## Exercise 4
Match searches with bookings  
• For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match.  
• For the bookings file, origin and destination are the columns dep_port and arr_port, respectively.  
• Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

• En este método se va a contar el número de reservas del fichero de booking por día, origen y destino. Después se va cruzar con searches para asignar el número de reservas por día, origen y destino. Por último se va a ir restando ese número de reservas según se va asignando 1 a cada registro hasta que el número de reservas quede a 0.  
• Se va a suponer que en el fichero searches cada búsqueda es para un único pasajero porque no tenemos el número de reservas por búsqueda.  
• Vamos a suponer que en el fichero de booking interesan todas las reservas, aunque luego se cancelen. Para eso se desechan los registros con pax < 1. Si se quisieran estudiar las reservas que acaban en vuelo sin cancelar habría que quitar ese filtro.

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None

In [2]:
# Se declaran las variables que tienen los ficheros SIN DUPLICADOS y el fichero de salida
searches_file = 'searches_nd.csv.bz2'
booking_file = 'booking_nd.csv.bz2'
searches_out = 'searches_bookings.csv'

In [3]:
# Se carga el DF de las búsquedas
searches_df = pd.read_csv(searches_file, sep='^', compression='bz2')
searches_df.columns = searches_df.columns.str.strip() # Se quitan los espacios en blanco de los nombres de las cols
searches_df.head()

/home/davidgp/anaconda3/envs/kschool/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [4]:
# Se carga el DF de las reservas
booking_df = pd.read_csv(booking_file, sep='^', compression='bz2')
booking_df.columns = booking_df.columns.str.strip() # Se quitan los espacios en blanco de los nombres de las cols
booking_df.head()

/home/davidgp/anaconda3/envs/kschool/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,8,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1.0,2013.0,3.0,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1.0,2013.0,3.0,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1.0,2013.0,3.0,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1.0,2013.0,3.0,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1.0,2013.0,3.0,SYDA82546


En el cruce entre ambos DF se van a usar las fechas de vuelo, aeropuerto de salida y aeropuerto de llegada. Para ello hay que preparar los datos:  
• Voy a suponer que el campo Seg1Date de searches y el campo brd_time de bookings tienen la fecha del vuelo.  
• Voy a ignorar la hora de embarque (brd_time) porque el campo Seg1Date no tiene hora.  
• En ambos DF voy a crear un campo para convertir las fechas de string a datetime.  
• En los campos de los aeropuertos se va a eliminar los espacios y se van a poner en mayúsculas

In [5]:
# Searches
# Se convierten  fechas la columna de brd_time, se eliminan espacios y se ponen mayúsculas
# para los aeropuertos de origen y destino
searches_df['Seg1Date2'] = pd.to_datetime(searches_df['Seg1Date'], format='%Y-%m-%d').dt.normalize()
searches_df['Origin2'] = searches_df['Origin'].str.strip().str.upper()
searches_df['Destination2'] = searches_df['Destination'].str.strip().str.upper()
searches_df.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Seg1Date2,Origin2,Destination2
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,2013-01-26,TXL,AUH
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,2013-01-04,ATH,MIL
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,2013-08-02,ICT,SFO
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,2013-01-02,RNB,ARN
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,2013-03-22,OSL,MAD


In [6]:
# Booking
# Antes de empezar eliminamos los registros con pasajeros negativos, porque son cancelaciones
# También se convierten  fechas la columna de brd_time, se eliminan espacios y se ponen mayúsculas
# para los aeropuertos de origen y destino
booking_df = booking_df[booking_df['pax'] > 0]
booking_df['brd_time2'] = pd.to_datetime(booking_df['brd_time'], format='%Y-%m-%d').dt.normalize()
booking_df['dep_port2'] = booking_df['dep_port'].str.strip().str.upper()
booking_df['arr_port2'] = booking_df['arr_port'].str.strip().str.upper()
booking_df.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid,brd_time2,dep_port2,arr_port2
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1.0,2013.0,3.0,NULL,2013-04-12,SAL,CLT
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1.0,2013.0,3.0,NULL,2013-07-15,SAL,CLT
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1.0,2013.0,3.0,SYDA82546,2013-04-24,AKL,SVO
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1.0,2013.0,3.0,SYDA82546,2013-05-14,AKL,SVO
5,2013-03-20 00:00:00,1V,US,0283592e45e392871f7e14b1e22a9aaf,2d0f8e7e1bb792625d2f34f221bc38f1,d54708298686de200f8270e7c637af5e,2013-03-20 00:00:00,5923,0,DEN,DEN,US,LGA,NYC,US,DEN,DEN,US,DEN,DEN,US,LGA,NYC,US,DENLGA,DENNYC,USUS,0,DENLGA,FK,W,Y,2013-04-07 10:23:00,2013-04-07 16:14:24,1.0,2013.0,3.0,NULL,2013-04-07,DEN,LGA


In [7]:
# Agregamos por día, salida y llegada para contar el número de búsqueda/día y reservas/día
# La agreación por búsqueda/día no es necesaria, pero nos permite analizar los datos.
# Por ejemplo, se ve que aunque las fechas de búsqueda son del año 2013, se buscan vuelos de 2014
searches_agg = searches_df.groupby(['Seg1Date2', 'Origin2', 'Destination2']).count()[['Seg1Date']]
searches_agg.columns = ['searches']
searches_agg[['searches']].sort_values(by='searches', ascending=False)

searches
Seg1Date2  Origin2 Destination2          
2014-06-28 NCE     LGW                 26
2013-06-28 NCE     LGW                 22
2013-12-09 STO     LON                 21
2013-07-06 BSB     POA                 19
2013-12-20 FRA     BKK                 14
...                                   ...
2013-07-07 JED     BEY                  1
                   AHB                  1
           JAX     LHR                  1
           JAI     BOM                  1
2014-12-22 SEA     ATL                  1

[342476 rows x 1 columns]

In [8]:
booking_agg = booking_df.groupby(['brd_time2', 'dep_port2', 'arr_port2']).count()[['brd_time']]
booking_agg.columns = ['bookings']
booking_agg[['bookings']].sort_values(by='bookings', ascending=False)

bookings
brd_time2  dep_port2 arr_port2          
2013-09-28 LHE       JED              79
2013-09-26 LHE       JED              75
           JED       LHE              32
2013-09-23 LHE       JED              28
2013-09-28 JED       LHE              24
...                                  ...
2013-04-20 CFE       NCE               1
           CEK       SVO               1
           CDG       YUL               1
                     TNR               1
2014-12-20 LHR       MIA               1

[565781 rows x 1 columns]

In [9]:
# Vamos a asignar el número de reservas a cada búsqueda. Primero eliminamos el índice de la tabla agregada
booking_agg.reset_index(inplace=True)
booking_agg.head()

,brd_time2,dep_port2,arr_port2,bookings
0,2012-03-02,AUS,MEX,1
1,2012-09-04,KWI,SKT,1
2,2012-10-01,BCN,GYE,1
3,2012-10-03,MIA,GIG,1
4,2012-10-04,SGN,YVR,1


In [10]:
# Se asigna a cada búsqueda el nº de búsquedas para ese día, salida y llegada
searches_df2 = searches_df.merge(booking_agg, 
                                 how='left', 
                                 left_on=['Seg1Date2','Origin2','Destination2'],
                                 right_on=['brd_time2','dep_port2','arr_port2'])

In [11]:
# Se ordenan los datos por día, origen y destino para después asignar un contador por grupo
searches_df2.sort_values(by=['Seg1Date2','Origin2','Destination2'], inplace=True)

In [12]:
# Se asigna un contador por cada día, origen, destino 
searches_df2['seq'] = searches_df2.groupby(['Seg1Date2', 'Origin2','Destination2']).cumcount()
# Se crea una columna que resta a cada reserva por día, origen y destino el contador de búsquedas 
searches_df2['seq2'] = searches_df2['bookings'] - searches_df2['seq']
# Se crea una columna que nos dice que la diferencia es mayor que 0 o no. En caso de ser mayor de 0 quiere decir
# que esa búsqueda se convierte en reserva. Como devuelve boolean convertimos a entero
searches_df2['end_in_booking'] = (searches_df2['seq2'] > 0).astype(int)

In [13]:
# Aquí hay un vuelo que tiene 11 búsquedas y 3 reservas, se ve muy bien como funciona el método
searches_df2[1990:2018]

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Seg1Date2,Origin2,Destination2,brd_time2,dep_port2,arr_port2,bookings,seq,seq2,end_in_booking
14831,2013-01-15,04:48:38,FCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,DMM,0.0,1.0,RUH,DMM,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,DMM,NaT,NaN,NaN,NaN,1,NaN,0
10697,2013-01-11,11:37:16,CAL,3565e31495ecfd46fa018339d20382b1,SA,RUH,DXB,1.0,2.0,RUH,DXB,2013-01-16,KP,NaN,DXB,RUH,2013-01-18,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,DXB,2013-01-16,RUH,DXB,3.0,0,3.0,1
2014,2013-01-03,10:55:33,FCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,0,3.0,1
2722,2013-01-03,12:45:19,FCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,1,2.0,1
6596,2013-01-07,06:39:59,CCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,2,1.0,1
6801,2013-01-07,22:44:58,FCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,3,0.0,0
7944,2013-01-08,14:01:07,CCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,4,-1.0,0
9832,2013-01-10,23:23:00,FCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,5,-2.0,0
12371,2013-01-13,14:34:25,FCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,6,-3.0,0
12411,2013-01-13,11:56:23,CCP,3565e31495ecfd46fa018339d20382b1,SA,RUH,JED,0.0,1.0,RUH,JED,2013-01-16,KP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,JED,2013-01-16,RUH,JED,2013-01-16,RUH,JED,3.0,7,-4.0,0


In [14]:
# Contamos el número de búsquedas convertidas en reserva y obtenemos el ratio
searches_df2['end_in_booking'].sum(), len(searches_df2['end_in_booking']), searches_df2['end_in_booking'].sum()/len(searches_df2['end_in_booking'])

(16844, 359003, 0.046918827976367886)

In [15]:
# Se limpian las columnas innecesarias antes de exportar
searches_df2.drop(['Seg1Date2', 'Origin2', 'Destination2', 'brd_time2', 
                   'dep_port2', 'arr_port2', 'bookings', 'seq', 'seq2'],
                  axis=1,
                  inplace=True)

In [16]:
searches_df2.columns

Index(['Date', 'Time', 'TxnCode', 'OfficeID', 'Country', 'Origin',
       'Destination', 'RoundTrip', 'NbSegments', 'Seg1Departure',
       'Seg1Arrival', 'Seg1Date', 'Seg1Carrier', 'Seg1BookingCode',
       'Seg2Departure', 'Seg2Arrival', 'Seg2Date', 'Seg2Carrier',
       'Seg2BookingCode', 'Seg3Departure', 'Seg3Arrival', 'Seg3Date',
       'Seg3Carrier', 'Seg3BookingCode', 'Seg4Departure', 'Seg4Arrival',
       'Seg4Date', 'Seg4Carrier', 'Seg4BookingCode', 'Seg5Departure',
       'Seg5Arrival', 'Seg5Date', 'Seg5Carrier', 'Seg5BookingCode',
       'Seg6Departure', 'Seg6Arrival', 'Seg6Date', 'Seg6Carrier',
       'Seg6BookingCode', 'From', 'IsPublishedForNeg', 'IsFromInternet',
       'IsFromVista', 'TerminalID', 'InternetOffice', 'end_in_booking'],
      dtype='object')

In [17]:
searches_df2.to_csv(searches_out, sep='^', index=False)